In [1]:
%cd /data/hpc/potato/ecoit_summer

/data/hpc/potato/ecoit_summer


In [5]:
import fitz  # PyMuPDF

def extract_text_from_pdf(pdf_path):
    text = ""
    with fitz.open(pdf_path) as doc:
        for page in doc:
            text += page.get_text("text")
    return text

ModuleNotFoundError: No module named 'fitz'

In [13]:
import pdfplumber
def iou(bbox1, bbox2, left=False):
    """
    Calculate the Intersection over Union (IoU) of two bounding boxes.
    bbox1 and bbox2 are tuples of (x0, y0, x1, y1).
    """
    x0 = max(bbox1[0], bbox2[0])
    y0 = max(bbox1[1], bbox2[1])
    x1 = min(bbox1[2], bbox2[2])
    y1 = min(bbox1[3], bbox2[3])

    intersection_area = max(0, x1 - x0) * max(0, y1 - y0)
    bbox1_area = (bbox1[2] - bbox1[0]) * (bbox1[3] - bbox1[1])
    bbox2_area = (bbox2[2] - bbox2[0]) * (bbox2[3] - bbox2[1])

    union_area = bbox1_area + bbox2_area - intersection_area if not left else bbox1_area
    return intersection_area / union_area if union_area > 0 else 0

def detect_paragraphs(pdf_path):
    paragraphs = []
    all_tables = []
    with pdfplumber.open(pdf_path) as pdf:
        print(len(pdf.pages))
        for page_number, page in enumerate(pdf.pages):
            # Extract text lines with their bounding boxes
            tables = page.find_tables()
            table_areas = []
            words = page.extract_words(x_tolerance= 0.5, 
                                        y_tolerance=1.35,
                                        keep_blank_chars=True, 
                                        use_text_flow=True)
            # If the page has tables, we can extract text from them
            for table_index, table in enumerate(tables):
                table_bbox = table.bbox
                table_areas.append(table_bbox)
                table_data = []
                for row in table.rows: 
                    row_data = []
                    print(row.cells)
                    for cell in row.cells:
                        print(cell)
                        if cell is None:
                            row_data.append("")
                            continue
                        cell_words = [w for w in words if iou([w['x0'], w['top'], w['x1'], w['bottom']], cell, left=True) >= 0.9]
                        cell_text = " ".join([w['text'] for w in cell_words])
                        row_data.append(cell_text)
                    table_data.append(row_data)
                all_tables.append(table_data)
            
            current_paragraph_words = []
            for i, word in enumerate(words):
                valid = True
                for bbox in table_areas:
                    word_bbox = [word['x0'], word['top'], word['x1'], word['bottom']]
                    # Check if the line intersects with any table area
                    if (iou(word_bbox, bbox, left=True) > 0.8):
                        # If it does, we skip adding this line to paragraphs
                        valid = False
                        break
                    else:
                        if "�" in word['text']:
                            print("Character not found")
                        print(word_bbox, bbox, iou(word_bbox, bbox, left=True))
                if not valid:
                    print(f"Skipping line due to table intersection: {word['text']}")
                    continue
                current_paragraph_words.append(word['text'])
                # Heuristic: Check for a significant vertical jump or end of page
                if i + 1 < len(words):
                    next_word = words[i+1]
                    # Adjust threshold based on typical line spacing in your PDFs
                    if next_word['top'] - word['bottom'] > 0.5 * word['height']:
                        paragraphs.append(" ".join(current_paragraph_words))
                        current_paragraph_words = []
                else: # End of page
                    paragraphs.append(" ".join(current_paragraph_words))

    return paragraphs, all_tables # Print the first 500 characters of the extracted text

In [16]:
pdf_text, tables = detect_paragraphs(r"data/database/Báo cáo/test_RMU.pdf")

71
[(128.3, 438.28999999999996, 158.06000000000003, 455.0899949999999), (158.06000000000003, 438.28999999999996, 197.56999999999996, 455.0899949999999), (197.56999999999996, 438.28999999999996, 250.48999999999995, 455.0899949999999), (250.48999999999995, 438.28999999999996, 305.8299910000001, 455.0899949999999), (305.8299910000001, 438.28999999999996, 350.8299910000001, 455.0899949999999), (350.8299910000001, 438.28999999999996, 418.6599910000001, 455.0899949999999), (418.6599910000001, 438.28999999999996, 501.21998999999977, 455.0899949999999)]
(128.3, 438.28999999999996, 158.06000000000003, 455.0899949999999)
(158.06000000000003, 438.28999999999996, 197.56999999999996, 455.0899949999999)
(197.56999999999996, 438.28999999999996, 250.48999999999995, 455.0899949999999)
(250.48999999999995, 438.28999999999996, 305.8299910000001, 455.0899949999999)
(305.8299910000001, 438.28999999999996, 350.8299910000001, 455.0899949999999)
(350.8299910000001, 438.28999999999996, 418.6599910000001, 455.0

In [15]:
for p in pdf_text:   
    print(p.replace(";", "\n").replace("  ", " ")) 

 
1 TẬP ĐOÀN ĐIỆN LỰC VIỆT NAM CỘNG HOÀ XÃ HỘI CHỦ NGHĨA VIỆT NAM 
TỔNG CÔNG TY ĐIỆN LỰC TP HÀ NỘI Độc lập - Tự do - Hạnh phúc 
         
THÔNG BÁO 
Về việc điều chỉnh tiêu chuẩn kỹ thuật tủ MU  Căn cứ thông tư quy định hệ thống điện phân phối số 3 9/2015/TT-BCT ngày 18/11/2015 của Bộ Công thương
 Căn cứ thông  o số 2325/TB-EVN HANOI ngày 23/05/2017 của Tổng Công ty Điện lực TP Hà Nội về việc  an hành tiêu chuẩn kỹ việc điều chỉnh,  ổ sung tiêu chuẩn kỹ thuật vật tư thiết ị cao, trung, hạ p
 Căn cứ vào tình hình vận hành đối với c c thiết  ị đang lắp trên lưới phân phối và c c công nghệ hiện tại
 
Tổng Công ty Điện lực TP Hà Nội thông  o tới c c đơn vị nội dung điều chỉnh,  ổ sung tiêu chuẩn kỹ thuật vật tư cao, trung, hạ  p trong c c quyết định v à thông báo sau: 1. TỦ  ING MAIN UNIT 35KV LOẠI T ONG NHÀ: KHÔNG MỞ  ỘNG ĐƯỢC, CÓ NGĂN SANG MÁY BIẾN ÁP DÙNG MÁY CẮT 
1.1. Phạm vi: 
- Yêu cầu kỹ thuật này ao gồm tủ RMU 35KV loại không mở rộng được trong nhà, được lắp r p và hiệu chỉnh tại n

IndexError: list index out of range

In [ ]:
import copy
executed = ['', '', '']
res_table = []
for row in tables[0]:
    row_data = [row[1], row[4], row[10]]
    row_data = ['' + data for data in row_data]
    if row_data[0] != '':
        if executed[0] != '' and row_data[0] != executed[0]:
            res_table.append(executed)
            executed = ['', '', '']

        if '.' not in row_data[0]:
            res_table.append(row_data)
            # executed = row_data
            continue
    
    for index, data in enumerate(row_data):
        if executed[index] != '' and data != '':
            executed[index] += ' '
        executed[index] += data

In [ ]:
import pdfplumber

pdf_path = r"C:\ecoit_summer\data\Mẫu phê duyệt tờ trình của KT (đơn giản)\03. 1phiếu trình.pdf"

full_text = ""
with pdfplumber.open(pdf_path) as pdf:
    for page in pdf.pages:
        # Extract text with layout (preserve whitespace)
        page_text = page.extract_text(x_tolerance=1, y_tolerance=1, layout=True)
        if page_text:
            full_text += page_text + "\n\n"

print(full_text)

In [20]:
import pdfminer
from pdfminer.layout import LAParams
from pdfminer.high_level import extract_text

pdf_path = r"data/database/Báo cáo/test_RMU.pdf"
laparams = LAParams(
    line_overlap=0.5,
    char_margin=2.0,
    line_margin=0.5,
    word_margin=0.1,
    boxes_flow=None,
    detect_vertical=False,
    all_texts=True
)

text=extract_text(pdf_file=open(pdf_path,'rb'),laparams=laparams)

In [21]:
print(text)

TẬP ĐOÀN ĐIỆN LỰC VIỆT NAM  CỘNG HOÀ XÃ HỘI CHỦ NGHĨA VIỆT NAM 

Độc lập - Tự do - Hạnh phúc 

TỔNG CÔNG TY 
ĐIỆN LỰC TP HÀ NỘI 

THÔNG BÁO 

Về việc điều chỉnh tiêu chuẩn kỹ thuật tủ  MU 

Căn  cứ  thông  tư  quy  định  hệ  thống  điện  phân  phối  số  39/2015/TT-BCT  ngày 

18/11/2015 của Bộ Công thương; 

Căn cứ thông   o số 2325/TB-EVN HANOI ngày 23/05/2017 của Tổng Công ty 
Điện lực TP Hà Nội về việc  an hành tiêu chuẩn kỹ  việc điều chỉnh,  ổ sung tiêu 
chuẩn kỹ thuật vật tư thiết  ị cao, trung, hạ  p; 

Căn cứ vào tình hình vận hành đối với c c thiết  ị đang lắp trên lưới phân phối 

và c c công nghệ hiện tại; 

Tổng Công ty Điện lực TP Hà Nội thông   o tới c c đơn vị nội dung điều chỉnh, 
 ổ sung tiêu chuẩn kỹ thuật vật tư cao, trung, hạ  p trong c c quyết định và thông 
báo sau: 
1. TỦ  ING MAIN UNIT 35KV LOẠI T ONG NHÀ: KHÔNG MỞ  ỘNG ĐƯỢC, 
CÓ NGĂN SANG MÁY BIẾN ÁP DÙNG MÁY CẮT 

1.1. Phạm vi:  

- Yêu cầu kỹ thuật này  ao gồm tủ RMU 35KV loại không mở rộng được trong nhà, 
đ

In [24]:
def transform_string(A: str, B: str):
    m, n = len(A), len(B)
    # dp[i][j] = minimum edits to transform A[:i] -> B[:j]
    dp = [[0] * (n + 1) for _ in range(m + 1)]

    # Initialize
    for i in range(m + 1):
        dp[i][0] = i  # delete all chars
    for j in range(n + 1):
        dp[0][j] = j  # insert all chars

    # Fill dp table
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            if A[i - 1] == B[j - 1]:
                dp[i][j] = dp[i - 1][j - 1]  # no operation
            else:
                dp[i][j] = 1 + min(
                    dp[i - 1][j],     # delete
                    dp[i][j - 1],     # insert
                    dp[i - 1][j - 1]  # replace
                )

    # Reconstruct operations
    ops = []
    i, j = m, n
    while i > 0 or j > 0:
        if i > 0 and j > 0 and A[i - 1] == B[j - 1]:
            i, j = i - 1, j - 1
        elif i > 0 and j > 0 and dp[i][j] == dp[i - 1][j - 1] + 1:
            ops.append(f"Replace '{A[i-1]}' at pos {i-1} with '{B[j-1]}'")
            i, j = i - 1, j - 1
        elif i > 0 and dp[i][j] == dp[i - 1][j] + 1:
            ops.append(f"Delete '{A[i-1]}' at pos {i-1}")
            i -= 1
        elif j > 0 and dp[i][j] == dp[i][j - 1] + 1:
            ops.append(f"Insert '{B[j-1]}' at pos {i}")
            j -= 1

    return dp[m][n], ops[::-1]


# Example
A = "Điện lực TP Hà Nội về việc an hành tiêu chuẩn kỹ việc điều chỉnh, ổ sung tiêu"
B = "Điện lực TP Hà Nội về việc ban hành tiêu chuẩn kỹ việc điều chinh, bổ sung tiêu"

dist, operations = transform_string(A, B)
print("Edit distance:", dist)
print("Operations:")
for op in operations:
    print(" -", op)

Edit distance: 3
Operations:
 - Insert 'b' at pos 27
 - Replace 'ỉ' at pos 61 with 'i'
 - Insert 'b' at pos 66
